In [6]:
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv
import os
from langgraph.prebuilt import tool_node
import google.generativeai as genai
import fitz
from typing import TypedDict
from pprint import pprint
import json

In [7]:
class AgentType(TypedDict):
    offer_analysis: dict
    preference_alignment: dict
    acceptance_prediction: dict
    upskill_recommendations: dict
    dashboard_summary: str

In [8]:
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel("gemini-1.5-flash")

In [9]:
def placement_dashboard_agent(state):
    offer = state["offer_analysis"]
    preference = state["preference_alignment"]
    prediction = state["acceptance_prediction"]
    upskill = state["upskill_recommendations"]

    # Parse if they are still JSON strings
    for k, v in [("offer_analysis", offer), 
                 ("preference_alignment", preference), 
                 ("acceptance_prediction", prediction), 
                 ("upskill_recommendations", upskill)]:
        if isinstance(v, str):
            try:
                state[k] = json.loads(v)
            except:
                pass

    # Re-extract after parsing
    offer = state["offer_analysis"]
    preference = state["preference_alignment"]
    prediction = state["acceptance_prediction"]
    upskill = state["upskill_recommendations"]

    summary = f"""
🔍 Candidate Offer Acceptance Report

📄 OFFER DETAILS:
- Job Title: {offer.get("job_title")}
- Salary: {offer.get("salary")}
- Location: {offer.get("location")}
- Joining Date: {offer.get("joining_date")}
- Bond Terms: {offer.get("bond_terms")}

🧑‍💼 CANDIDATE PREFERENCES:
- Preferred Location: {preference.get("preferred_location")}
- Interests: {", ".join(preference.get("interests", []))}
- Skills: {", ".join(preference.get("skills", []))}
- Expected CTC: {preference.get("expected_ctc")}
- Career Goals: {preference.get("career_goals")}

📈 ACCEPTANCE PREDICTION:
- Likelihood: {prediction.get("acceptance_likelihood")}%
- Reason: {prediction.get("reasoning")}

🎯 UPSKILLING RECOMMENDATIONS:
- Skill Gap: {upskill.get("skill_gap_analysis")}
- Recommended Courses: {", ".join(upskill.get("recommended_courses", []))}
- Onboarding Resources: {", ".join(upskill.get("onboarding_resources", []))}

✅ ACTIONABLE INSIGHT:
The candidate is likely to accept the offer. Recommend initiating onboarding steps and sharing recommended courses.

"""
    return {"dashboard_summary": summary.strip()}


In [10]:
builder = StateGraph(AgentType)

builder.add_node("DashboardAgent", placement_dashboard_agent)
builder.set_entry_point("DashboardAgent")
builder.set_finish_point("DashboardAgent")

graph = builder.compile()


In [11]:
result = graph.invoke({
    "offer_analysis": {'bond_terms': None,
 'job_title': 'Software Engineer',
 'joining_date': '2017-05-18',
 'location': 'Nasik',
 'salary': {'gross_annual': 314935.38,
            'monthly_breakdown': {'basic': 8000,
                                  'bonus': 6300,
                                  'conveyance': 3600,
                                  'education_allowance': 3600,
                                  'gratuity': 4615,
                                  'hra': 4800,
                                  'leave_benefit': 14000,
                                  'medical_reimbursement': 1500,
                                  'mediclaim': 1000,
                                  'pf': 960,
                                  'variable': 3000},
            'variable_component': 'Based on monthly PPI (Personal Performance '
                                  'Index) + CPI (Company Performance Index)'}},
    "preference_alignment": {'career_goals': 'Contribute to innovative projects that make a difference',
 'expected_ctc': None,
 'interests': ['Full Stack Development',
               'MERN Stack',
               'PERN Stack',
               'Scalable Web Applications',
               'Innovative Projects'],
 'preferred_location': 'Coimbatore, Tamil Nadu, India',
 'skills': ['JavaScript',
            'Nodejs',
            'fastAPI',
            'Python',
            'Express js',
            'MongoDB',
            'PostgreSQL',
            'Redis',
            'Elasticsearch',
            'RabbitMQ',
            'React js',
            'Next js',
            'Amazon S3',
            'AWS Elastic Beanstalk',
            'AWS Cloudfront',
            'Prisma ORM',
            'UI/UX',
            'Responsive Web Design']},
    "acceptance_prediction": {'acceptance_likelihood': 30,
 'reasoning': "The candidate's strong preference for Coimbatore significantly "
              'lowers the likelihood of acceptance.  The Nasik location is a '
              "major drawback. While the salary is reasonable,  there's no "
              "information on the specifics of the projects or the company's "
              "culture, both of which are important given the candidate's "
              "focus on innovative projects. The candidate's skills align with "
              'the Software Engineer role, but the location mismatch is the '
              'dominant factor in predicting a low probability of acceptance.'},
    "upskill_recommendations": {'onboarding_resources': ['Company Internal Wiki/Documentation',
                          'Code Style Guide',
                          'Project Management Tool Tutorial (e.g., Jira, '
                          'Asana)',
                          'Git Workflow and Best Practices Guide',
                          'Introduction to the Company Culture and Values',
                          'Team Introduction and Communication Channels',
                          'Onboarding Checklist and Timeline'],
 'recommended_courses': ['Advanced React.js and Redux',
                         'Mastering Next.js',
                         'Building Scalable APIs with Node.js and Express.js',
                         'Microservices Architecture with Node.js',
                         'Optimizing Database Performance with PostgreSQL',
                         'AWS Certified Developer - Associate'],
 'skill_gap_analysis': 'While the candidate possesses a strong foundation in '
                       'full-stack development with experience in MERN and '
                       'PERN stacks,  further specialization in areas like '
                       'advanced React.js, Redux, Next.js optimization and '
                       'deeper understanding of microservices architecture '
                       'would be beneficial.  More comprehensive AWS '
                       'knowledge, especially beyond Elastic Beanstalk and '
                       'Cloudfront, is also recommended.  The candidate’s '
                       'skillset aligns well with the job requirements but '
                       'targeted training will enhance their efficiency and '
                       'contributions.'}
})

print(result["dashboard_summary"])


🔍 Candidate Offer Acceptance Report

📄 OFFER DETAILS:
- Job Title: Software Engineer
- Salary: {'gross_annual': 314935.38, 'monthly_breakdown': {'basic': 8000, 'bonus': 6300, 'conveyance': 3600, 'education_allowance': 3600, 'gratuity': 4615, 'hra': 4800, 'leave_benefit': 14000, 'medical_reimbursement': 1500, 'mediclaim': 1000, 'pf': 960, 'variable': 3000}, 'variable_component': 'Based on monthly PPI (Personal Performance Index) + CPI (Company Performance Index)'}
- Location: Nasik
- Joining Date: 2017-05-18
- Bond Terms: None

🧑‍💼 CANDIDATE PREFERENCES:
- Preferred Location: Coimbatore, Tamil Nadu, India
- Interests: Full Stack Development, MERN Stack, PERN Stack, Scalable Web Applications, Innovative Projects
- Skills: JavaScript, Nodejs, fastAPI, Python, Express js, MongoDB, PostgreSQL, Redis, Elasticsearch, RabbitMQ, React js, Next js, Amazon S3, AWS Elastic Beanstalk, AWS Cloudfront, Prisma ORM, UI/UX, Responsive Web Design
- Expected CTC: None
- Career Goals: Contribute to innovat